# DataInsights #1/session2

- make a clean data set for volunteers to look at in second session

This cleans the WALDEC data set from: https://www.data.gouv.fr/fr/datasets/repertoire-national-des-associations/

#### how many lines does original csv have?

I wonder how many we are missing?

In [1]:
path = './../../data/rna_waldec_20191001/rna_waldec_20191001.csv'

import csv

with open(path) as csvfile:
    reader = csv.DictReader(csvfile)
    
    len(list(reader))

#### Load data with pandas

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(path, sep=';', error_bad_lines=False, engine='python')

In [4]:
df.shape

(1733433, 39)

In [5]:
df.head(3)

,id,id_ex,siret,rup_mi,gestion,date_creat,date_decla,date_publi,date_disso,nature,...,adrg_distrib,adrg_codepostal,adrg_achemine,adrg_pays,dir_civilite,siteweb,publiweb,observation,position,maj_time
0,W632007046,NaN,NaN,NaN,632P,2014-06-17,2015-01-21,2014-06-28,2015-01-20,D,...,NaN,63000,CLERMONT-FERRAND,FRANCE,PP,NaN,0.0,NaN,D,2015-02-09 12:51:34
1,W012000002,2/08825,NaN,NaN,691P,1983-10-10,2004-03-11,0001-01-01,0001-01-01,D,...,NaN,69100,villeurbanne,FRANCE,PM,NaN,1.0,NaN,A,2008-02-12 14:40:18
2,W012000005,0012011852,NaN,NaN,691P,2003-09-01,2005-01-25,0001-01-01,0001-01-01,D,...,NaN,69270,FONTAINE SUR SAONE,FRANCE,PM,NaN,1.0,NaN,A,2007-04-05 09:41:02


#### find proportion missing from each column

In [6]:
pd.DataFrame(df.isnull().sum()/df.shape[0]).sort_values(by=0, ascending=False).head(20)

,0
adrs_repetition,0.998324
rup_mi,0.996639
siret,0.981590
adrg_declarant,0.976880
siteweb,0.968481
adrg_complemgeo,0.948997
adrg_distrib,0.906085
observation,0.783862
adrg_complemid,0.733536
adrs_complement,0.662397


#### dropcolumns that are not of use

There are many missing values in many columns:

In [7]:
cols_lots_missing_values = [
#     'adrs_repetition',
    'rup_mi',
    'siret',
#     'adrg_declarant',
    'siteweb',
#     'adrg_complemgeo',
#     'adrg_distrib',
]

Some columns look non-useful:

In [8]:
cols_notinteresting = [
    'observation', # this seems to be notes about the admin on the database
    'id_ex', # this is an old ref, 50% are missing, there is a newer one 'id'
    'date_decla', # we have date created which will do, same for col below
    'date_publi',
    'gestion', #where the associations were recored, presumably online/prefecture.
    'publiweb', # can publish on web or not
    'maj_time', # time aded to database
]


In [9]:
cols_todrop = cols_lots_missing_values + cols_notinteresting
dff = df.drop(cols_todrop,axis=1)

In [10]:
dff.shape

(1733433, 29)

In [11]:
cols_todrop

['rup_mi',
 'siret',
 'siteweb',
 'observation',
 'id_ex',
 'date_decla',
 'date_publi',
 'gestion',
 'publiweb',
 'maj_time']

In [12]:
for  i in dff.columns:
    print(i)

id
date_creat
date_disso
nature
groupement
titre
titre_court
objet
objet_social1
objet_social2
adrs_complement
adrs_numvoie
adrs_repetition
adrs_typevoie
adrs_libvoie
adrs_distrib
adrs_codeinsee
adrs_codepostal
adrs_libcommune
adrg_declarant
adrg_complemid
adrg_complemgeo
adrg_libvoie
adrg_distrib
adrg_codepostal
adrg_achemine
adrg_pays
dir_civilite
position


#### merge table of objet_social1, and objet_social2 so have text description

In [13]:
objet_desc = pd.read_excel('./../../data/Fichier_RNA_-_Nomenclature_Complete_Objet_Social_.xlsx')

There are some address values in the objet columns in the waldoc data set. Clean those and replace with Nans now.

In [14]:
poss_values = objet_desc['Code Objet Social'].values

def filter_func(x):
    ""
    value = np.nan
    if len(str(x)) == 6:
        value=x
    return(value)

# filter_func(11111)

dff.objet_social1 = dff.objet_social1.apply(filter_func)
dff.objet_social2 = dff.objet_social2.apply(filter_func)

In [15]:
objet_desc['Code Objet Social'].dtype

dtype('int64')

In [16]:
dff.objet_social1.isnull().sum()

113

There are some missing values in objet_social1 & 2. Dropping these rows as wont merge otherwise. We lose about 300 associaitons.

In [17]:
dff.shape

(1733433, 29)

In [18]:
dff.dropna(subset=['objet_social1','objet_social2'],inplace=True)

dff.shape

(1733101, 29)

In [19]:
erroneous_values = ['40 000','023àà(','011.75']

dff.drop(index = dff[dff.objet_social2.isin( erroneous_values)].index, inplace=True)

In [20]:
dff.shape

(1733098, 29)

In [21]:
dff.objet_social1 = dff.objet_social1.astype(int)
dff.objet_social2 = dff.objet_social2.astype(int)

In [22]:
dff = dff.merge(objet_desc, how='left', left_on='objet_social1', right_on='Code Objet Social')
dff.drop(columns=['Code Objet Social'],inplace=True)
dff.rename({'Objet Social':'objet_social1_desc'},axis=1, inplace=True)
dff.shape

(1733098, 30)

In [23]:
dff = dff.merge(objet_desc, how='left', left_on='objet_social2', right_on='Code Objet Social')
dff.drop(columns=['Code Objet Social'],inplace=True)
dff.rename({'Objet Social':'objet_social2_desc'}, axis=1, inplace=True)
dff.shape

(1733098, 31)

#### create flag for association no longer active 


In [24]:
dff['association_active'] = 0
dff.loc[dff.position == 'A', 'association_active'] = 1

In [25]:
dff.association_active.value_counts()

1    1531245
0     201853
Name: association_active, dtype: int64

#### check through columns and check how many lines are messed up with wrong types of data

If there are not too many then we just delete them i think.
- code_postal, len 5
- objet_social1, len 6

#### clean adress fields

## export data to csv

- put columns in nice order

In [26]:
dff.rename({'Objet Social_x':'objet_social2_desc'},axis=1, inplace=True)

In [27]:
dff.head()

,id,date_creat,date_disso,nature,groupement,titre,titre_court,objet,objet_social1,objet_social2,...,adrg_libvoie,adrg_distrib,adrg_codepostal,adrg_achemine,adrg_pays,dir_civilite,position,objet_social1_desc,objet_social2_desc,association_active
0,W632007046,2014-06-17,2015-01-20,D,S,BLACK OWL,BLACK OWL,"promouvoir la scène indépendante musicale ""psy...",6030,0,...,35 RUE PASCAL,NaN,63000,CLERMONT-FERRAND,FRANCE,PP,D,"chant choral, musique",NaN,0
1,W012000002,1983-10-10,0001-01-01,D,S,ARIADNE,ARIADNE,Favoriser le développement d'une compagnie thé...,6070,6070,...,39 rue Courteline,NaN,69100,villeurbanne,FRANCE,PM,A,"théâtre, marionnettes, cirque, spectacles de v...","théâtre, marionnettes, cirque, spectacles de v...",1
2,W012000005,2003-09-01,0001-01-01,D,S,UNITE FRANCAISE D'INTERVENTION EN CATASTROPHE ...,UNITE FRANCAISE D'INTERVENTION EN ...,Intervenir lors de catastrophes naturelles sur...,20000,20000,...,18 rue Ampère,NaN,69270,FONTAINE SUR SAONE,FRANCE,PM,A,"associations caritatives, humanitaires, aide a...","associations caritatives, humanitaires, aide a...",1
3,W012000006,1999-02-11,2008-02-27,D,S,TRIADE BASS,TRIADE BASSE,la promotion et la diffusion de la création ar...,6000,6000,...,44 QUAI CLEMENCEAU,NaN,69300,CALUIRE ET CUIRE,FRANCE,PM,D,"culture, pratiques d\’activités artistiques, c...","culture, pratiques d\’activités artistiques, c...",0
4,W033002617,2008-05-19,0001-01-01,D,S,ASSOCIATION MAYOTTE FOOTBALL CLUB DE BELLERIVE...,MAYOTTE FOOTBALL CLUB BSA,pratiquer et développer le football; organisat...,11075,0,...,8 RUE EMMANUEL CHABRIER,NaN,03700,BELLERIVE-SUR-ALLIER,FRANCE,PM,A,"Football (football, futsal)",NaN,1


In [28]:
dff.columns

Index(['id', 'date_creat', 'date_disso', 'nature', 'groupement', 'titre',
       'titre_court', 'objet', 'objet_social1', 'objet_social2',
       'adrs_complement', 'adrs_numvoie', 'adrs_repetition', 'adrs_typevoie',
       'adrs_libvoie', 'adrs_distrib', 'adrs_codeinsee', 'adrs_codepostal',
       'adrs_libcommune', 'adrg_declarant', 'adrg_complemid',
       'adrg_complemgeo', 'adrg_libvoie', 'adrg_distrib', 'adrg_codepostal',
       'adrg_achemine', 'adrg_pays', 'dir_civilite', 'position',
       'objet_social1_desc', 'objet_social2_desc', 'association_active'],
      dtype='object')

In [29]:
dff.to_csv('./../../data_processed/clean_data.csv', index=False)